In [1]:
import pandas as pd
import numpy as np
import os
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import __version__ as sklearn_version
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, learning_curve
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectKBest, f_regression
import datetime

In [3]:
opera = pd.read_excel('/Users/angelique/Documents/GitHub/New_Opera_Company_Capstone/Excel and CSV Files/opera_4.xlsx')
opera.head()

,season,iso,city,composer,db,dd,nat,mf,work,worknat,...,performances_season_by_city,perf_per_1k_ppl_city_pop,opera_by_composer,performances_season_by_country_total,performances_season_by_city_total,perf_total_per_1k_city_pop,perf_total_per_10k_co_pop,Season Year,country_change_from_previous_season,city_change_from_previous_season
0,1213,al,Tirana,Lortzing,1801,1851,de,m,Ali Pascha von Janina,de,...,9,0.021506,Ali Pascha von Janina by Lor,110,110,0.262847,0.385301,2013-01-01,NaN,NaN
1,1213,al,Tirana,Mozart,1756,1791,at,m,Don Giovanni,it,...,9,0.021506,Don Giovanni by Moz,110,110,0.262847,0.385301,2013-01-01,NaN,NaN
2,1213,al,Tirana,Puccini,1858,1924,it,m,Tosca,it,...,9,0.021506,Tosca by Puc,110,110,0.262847,0.385301,2013-01-01,NaN,NaN
3,1213,am,Yerevan,Spendiaryan,1871,1928,am,m,Almast,am,...,5,0.004573,Almast by Spe,111,111,0.101510,0.371147,2013-01-01,NaN,NaN
4,1213,am,Yerevan,Tigranian,1879,1950,am,m,Anoush,am,...,5,0.004573,Anoush by Tig,111,111,0.101510,0.371147,2013-01-01,NaN,NaN


In [4]:
opera.columns

Index(['season', 'iso', 'city', 'composer', 'db', 'dd', 'nat', 'mf', 'work',
       'worknat', 'type', 'start date', 'performances', 'Country Name',
       'city population', 'country population', 'continent', 'sub-region',
       'performances_season_by_country', 'perf_per_10k_ppl_co_pop',
       'performances_season_by_city', 'perf_per_1k_ppl_city_pop',
       'opera_by_composer', 'performances_season_by_country_total',
       'performances_season_by_city_total', 'perf_total_per_1k_city_pop',
       'perf_total_per_10k_co_pop', 'Season Year',
       'country_change_from_previous_season',
       'city_change_from_previous_season'],
      dtype='object')

In [5]:
opera.dtypes

season                                           int64
iso                                             object
city                                            object
composer                                        object
db                                              object
dd                                              object
nat                                             object
mf                                              object
work                                            object
worknat                                         object
type                                            object
start date                              datetime64[ns]
performances                                     int64
Country Name                                    object
city population                                  int64
country population                             float64
continent                                       object
sub-region                                      object
performanc

In [9]:
opera['Year'] = opera['start date'].dt.year
opera['Month'] = opera['start date'].dt.month
opera['Day'] = opera['start date'].dt.day
opera['Weekday'] = opera['start date'].dt.weekday
opera['Week_of_Year'] = opera['start date'].dt.isocalendar().week
opera['Days_Since_Start'] = (opera['start date'] - opera['start date'].min()).dt.days
opera.head()

,season,iso,city,composer,db,dd,nat,mf,work,worknat,...,perf_total_per_10k_co_pop,Season Year,country_change_from_previous_season,city_change_from_previous_season,Year,Month,Day,Weekday,Week_of_Year,Days_Since_Start
0,1213,al,Tirana,Lortzing,1801,1851,de,m,Ali Pascha von Janina,de,...,0.385301,2013-01-01,NaN,NaN,2013,3,23,5,12,273
1,1213,al,Tirana,Mozart,1756,1791,at,m,Don Giovanni,it,...,0.385301,2013-01-01,NaN,NaN,2013,5,18,5,20,329
2,1213,al,Tirana,Puccini,1858,1924,it,m,Tosca,it,...,0.385301,2013-01-01,NaN,NaN,2013,2,13,2,7,235
3,1213,am,Yerevan,Spendiaryan,1871,1928,am,m,Almast,am,...,0.371147,2013-01-01,NaN,NaN,2013,7,11,3,28,383
4,1213,am,Yerevan,Tigranian,1879,1950,am,m,Anoush,am,...,0.371147,2013-01-01,NaN,NaN,2013,5,11,5,19,322


In [11]:
opera.columns

Index(['season', 'iso', 'city', 'composer', 'db', 'dd', 'nat', 'mf', 'work',
       'worknat', 'type', 'start date', 'performances', 'Country Name',
       'city population', 'country population', 'continent', 'sub-region',
       'performances_season_by_country', 'perf_per_10k_ppl_co_pop',
       'performances_season_by_city', 'perf_per_1k_ppl_city_pop',
       'opera_by_composer', 'performances_season_by_country_total',
       'performances_season_by_city_total', 'perf_total_per_1k_city_pop',
       'perf_total_per_10k_co_pop', 'Season Year',
       'country_change_from_previous_season',
       'city_change_from_previous_season', 'Year', 'Month', 'Day', 'Weekday',
       'Week_of_Year', 'Days_Since_Start'],
      dtype='object')

In [13]:
# Rank cities based on the sum of growth in performances
opera_city_growth = opera[['city', 'city_change_from_previous_season', 'iso']]

# Group by city and iso, and sum the 'city_change_from_previous_season' for each city
city_growth_sum = opera_city_growth.groupby(['city', 'iso'], as_index=False)['city_change_from_previous_season'].sum()

# Sort cities by total growth (sum), highest growth first
city_growth_sorted = city_growth_sum.sort_values(by='city_change_from_previous_season', ascending=False)

# Drop duplicates and keep the row with the highest growth for each city
opera_city_growth_unique = city_growth_sorted.drop_duplicates(subset='city', keep='first')

# Get the top cities with the highest total growth
top_cities = opera_city_growth_unique.head(25)  # Top 10 cities with the highest total growth

# Print the result
print(top_cities)

               city iso  city_change_from_previous_season
748          Moscow  ru                           27911.0
1216     Washington  us                           16128.0
1062  St Petersburg  ru                            9681.0
51        Arlington  us                            9648.0
263        Columbus  us                            3549.0
759         München  de                            3381.0
931        Richmond  us                            3042.0
846           Paris  fr                            2971.0
868    Philadelphia  us                            2916.0
327      Dusseldorf  de                            1677.0
315         Dresden  de                            1674.0
892     Portland OR  us                            1539.0
62          Atlanta  us                            1312.0
295          Denver  us                            1168.0
371         Firenze  it                            1163.0
1204       Voronezh  ru                             979.0
290          D

In [15]:
# Rank countries based on the sum of growth in performances
opera_country_growth = opera[['Country Name', 'country_change_from_previous_season', 'iso']]

# Group by country and iso, and sum the 'country_change_from_previous_season' for each country
country_growth_sum = opera_country_growth.groupby(['Country Name', 'iso'], as_index=False)['country_change_from_previous_season'].sum()

# Sort countries by total growth (sum), highest growth first
country_growth_sorted = country_growth_sum.sort_values(by='country_change_from_previous_season', ascending=False)

# Drop duplicates and keep the row with the highest growth for each city
opera_country_growth_unique = country_growth_sorted.drop_duplicates(subset='Country Name', keep='first')

# Get the top cities with the highest total growth
top_countries = opera_country_growth_unique.head(25)  # Top 10 cities with the highest total growth

# Print the result
print(top_countries)

          Country Name iso  country_change_from_previous_season
58  Russian Federation  ru                             194425.0
34               Italy  it                              87941.0
65              Sweden  se                               6459.0
55              Poland  pl                               5653.0
64               Spain  es                               3115.0
50         Netherlands  nl                               2892.0
35               Japan  jp                                631.0
39              Latvia  lv                                529.0
23             Estonia  ee                                517.0
62            Slovenia  si                                509.0
6              Belarus  by                                450.0
73          Uzbekistan  uz                                450.0
2              Armenia  am                                442.0
13               China  cn                                396.0
59              Serbia  rs              

In [17]:
nan_count_per_column = opera.isna().sum()
print(nan_count_per_column)

season                                     0
iso                                        0
city                                       0
composer                                   5
db                                       227
dd                                      3593
nat                                       11
mf                                         0
work                                       5
worknat                                 1548
type                                       0
start date                                 0
performances                               0
Country Name                               0
city population                            0
country population                         0
continent                                  0
sub-region                                 0
performances_season_by_country             0
perf_per_10k_ppl_co_pop                    0
performances_season_by_city                0
perf_per_1k_ppl_city_pop                   0
opera_by_c

In [19]:
# Step 1: Select only numeric columns
numeric_columns = opera.select_dtypes(include=[np.number])

# Step 2: Check for infinity values in the numeric columns
infinity_counts = np.isinf(numeric_columns).sum()

# Step 3: Print the count of infinity values per numeric column
print("Infinity counts per numeric column:")
print(infinity_counts)

Infinity counts per numeric column:
season                                  0
performances                            0
city population                         0
country population                      0
performances_season_by_country          0
perf_per_10k_ppl_co_pop                 0
performances_season_by_city             0
perf_per_1k_ppl_city_pop                0
performances_season_by_country_total    0
performances_season_by_city_total       0
perf_total_per_1k_city_pop              0
perf_total_per_10k_co_pop               0
country_change_from_previous_season     0
city_change_from_previous_season        0
Year                                    0
Month                                   0
Day                                     0
Weekday                                 0
Week_of_Year                            0
Days_Since_Start                        0
dtype: Int64


In [21]:
# Define the condition for selecting rows
condition = (opera['iso'] == 'cu') & (opera['city'] == 'La Habana')

# Apply fillna() to the 'country_change_from_previous_season' column for the selected rows
opera.loc[condition, 'country_change_from_previous_season'] = opera.loc[
    condition, 'country_change_from_previous_season'
].fillna(2)

In [23]:
# Display rows where 'iso' is 'cu' and 'city' is 'La Habana'
print(opera[(opera['iso'] == 'cu') & (opera['city'] == 'La Habana')])

       season iso       city composer    db    dd nat mf          work  \
27083    1617  cu  La Habana   Mozart  1756  1791  at  m  Don Giovanni   

      worknat  ... perf_total_per_10k_co_pop Season Year  \
27083      it  ...                  0.001783  2017-01-01   

       country_change_from_previous_season city_change_from_previous_season  \
27083                                  2.0                              NaN   

       Year  Month Day Weekday  Week_of_Year  Days_Since_Start  
27083  2016     12  17       5            50              1638  

[1 rows x 36 columns]


In [25]:
# Step 1: Identify rows with missing values in 'country_change_from_previous_season'
country_missing = opera[opera['country_change_from_previous_season'].isna()]

# Step 2: Use rows without missing values to train the model
country_not_missing = opera.dropna(subset=['country_change_from_previous_season'])

# Separate features (X) and target (y) for rows without missing values
X_no_missing = country_not_missing[['city population', 'country population', 'performances_season_by_country', 
                                    'perf_per_10k_ppl_co_pop', 'performances_season_by_city', 'perf_per_1k_ppl_city_pop', 
                                    'Year']]
y_no_missing = country_not_missing['country_change_from_previous_season']

# Step 3: Train the Linear Regression Model for each iso
predicted_values = []

# Group the data by 'iso' and perform the prediction for each group
for iso_code, group in country_not_missing.groupby('iso'):
    # Separate the group into features (X) and target (y)
    X_group = group[['city population', 'country population', 'performances_season_by_country', 
                     'perf_per_10k_ppl_co_pop', 'performances_season_by_city', 'perf_per_1k_ppl_city_pop', 'Year']]
    y_group = group['country_change_from_previous_season']
    
    # Train the linear regression model for this specific iso code
    model = LinearRegression()
    model.fit(X_group, y_group)
    
    # Get the rows for this iso code that have missing values
    missing_group = country_missing[country_missing['iso'] == iso_code]
    
    # If there are any missing values for this iso, predict them
    if not missing_group.empty:
        X_missing = missing_group[['city population', 'country population', 'performances_season_by_country', 
                                  'perf_per_10k_ppl_co_pop', 'performances_season_by_city', 'perf_per_1k_ppl_city_pop', 'Year']]
        
        # Predict the missing values
        predictions = model.predict(X_missing)
        
        # Store the predicted values and add to the list
        predicted_values.extend(predictions)

# Step 4: Check the lengths to confirm they match
print(f"Length of country_missing: {len(country_missing)}")
print(f"Length of predicted values: {len(predicted_values)}")


Length of country_missing: 6558
Length of predicted values: 6558


In [27]:
# Step 5: Assign the predicted values to the original DataFrame
country_missing['country_change_from_previous_season'] = predicted_values

# Print the updated dataframe with predicted values
print(country_missing)

       season iso     city        composer    db    dd nat mf  \
0        1213  al   Tirana        Lortzing  1801  1851  de  m   
1        1213  al   Tirana          Mozart  1756  1791  at  m   
2        1213  al   Tirana         Puccini  1858  1924  it  m   
3        1213  am  Yerevan     Spendiaryan  1871  1928  am  m   
4        1213  am  Yerevan       Tigranian  1879  1950  am  m   
...       ...  ..      ...             ...   ...   ...  .. ..   
10091    1314  kz   Astana           Verdi  1813  1901  it  m   
10240    1314  mo    Macau          Mozart  1756  1791  at  m   
10241    1314  mo    Macau           Verdi  1813  1901  it  m   
10242    1314  mo    Macau           Verdi  1813  1901  it  m   
10243    1314  mo    Macau  Wagner,Richard  1813  1883  de  m   

                        work worknat  ... perf_total_per_10k_co_pop  \
0      Ali Pascha von Janina      de  ...                  0.385301   
1               Don Giovanni      it  ...                  0.385301   
2     

/var/folders/v8/j1kjtws546j8drwljwyklgrm0000gn/T/ipykernel_6184/1160055947.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  country_missing['country_change_from_previous_season'] = predicted_values


In [29]:
# Step 5: Fill the missing values with the predicted values
opera.loc[opera['country_change_from_previous_season'].isna(), 'country_change_from_previous_season'] = predicted_values

# Now 'df' has the missing values in 'country_change_from_previous_season' filled
print(opera.head())

   season iso     city     composer    db    dd nat mf                   work  \
0    1213  al   Tirana     Lortzing  1801  1851  de  m  Ali Pascha von Janina   
1    1213  al   Tirana       Mozart  1756  1791  at  m           Don Giovanni   
2    1213  al   Tirana      Puccini  1858  1924  it  m                  Tosca   
3    1213  am  Yerevan  Spendiaryan  1871  1928  am  m                 Almast   
4    1213  am  Yerevan    Tigranian  1879  1950  am  m                 Anoush   

  worknat  ... perf_total_per_10k_co_pop Season Year  \
0      de  ...                  0.385301  2013-01-01   
1      it  ...                  0.385301  2013-01-01   
2      it  ...                  0.385301  2013-01-01   
3      am  ...                  0.371147  2013-01-01   
4      am  ...                  0.371147  2013-01-01   

   country_change_from_previous_season city_change_from_previous_season  Year  \
0                             2.230995                              NaN  2013   
1             

In [31]:
nan_count_per_column = opera.isna().sum()
print(nan_count_per_column)

season                                     0
iso                                        0
city                                       0
composer                                   5
db                                       227
dd                                      3593
nat                                       11
mf                                         0
work                                       5
worknat                                 1548
type                                       0
start date                                 0
performances                               0
Country Name                               0
city population                            0
country population                         0
continent                                  0
sub-region                                 0
performances_season_by_country             0
perf_per_10k_ppl_co_pop                    0
performances_season_by_city                0
perf_per_1k_ppl_city_pop                   0
opera_by_c

In [33]:
# Define the condition for selecting rows
condition = (opera['iso'] == 'cu') & (opera['city'] == 'La Habana')

# Apply fillna() to the 'country_change_from_previous_season' column for the selected rows
opera.loc[condition, 'city_change_from_previous_season'] = opera.loc[
    condition, 'city_change_from_previous_season'
].fillna(2)

In [35]:
# Step 1: Identify rows with missing values in 'city_change_from_previous_season'
city_missing = opera[opera['city_change_from_previous_season'].isna()]

# Step 2: Use rows without missing values to train the model
city_not_missing = opera.dropna(subset=['city_change_from_previous_season'])

# Separate features (X) and target (y) for rows without missing values
X_no_missing = city_not_missing[['city population', 'country population', 'performances_season_by_country', 
                                    'perf_per_10k_ppl_co_pop', 'performances_season_by_city', 'perf_per_1k_ppl_city_pop', 
                                    'Year']]
y_no_missing = city_not_missing['city_change_from_previous_season']

# Step 3: Train the Linear Regression Model for each iso
predicted_values = []

# Group the data by 'iso' and perform the prediction for each group
for iso_code, group in city_not_missing.groupby('iso'):
    # Separate the group into features (X) and target (y)
    X_group = group[['city population', 'country population', 'performances_season_by_country', 
                     'perf_per_10k_ppl_co_pop', 'performances_season_by_city', 'perf_per_1k_ppl_city_pop', 'Year']]
    y_group = group['city_change_from_previous_season']
    
    # Train the linear regression model for this specific iso code
    model = LinearRegression()
    model.fit(X_group, y_group)
    
    # Get the rows for this iso code that have missing values
    missing_group = city_missing[city_missing['iso'] == iso_code]
    
    # If there are any missing values for this iso, predict them
    if not missing_group.empty:
        X_missing = missing_group[['city population', 'country population', 'performances_season_by_country', 
                                  'perf_per_10k_ppl_co_pop', 'performances_season_by_city', 'perf_per_1k_ppl_city_pop', 'Year']]
        
        # Predict the missing values
        predictions = model.predict(X_missing)
        
        # Store the predicted values and add to the list
        predicted_values.extend(predictions)

# Step 4: Check the lengths to confirm they match
print(f"Length of city_missing: {len(city_missing)}")
print(f"Length of predicted values: {len(predicted_values)}")

Length of city_missing: 7190
Length of predicted values: 7190


In [37]:
# Step 5: Fill the missing values with the predicted values
opera.loc[opera['city_change_from_previous_season'].isna(), 'city_change_from_previous_season'] = predicted_values

# Now 'df' has the missing values in 'country_change_from_previous_season' filled
print(opera.head())

   season iso     city     composer    db    dd nat mf                   work  \
0    1213  al   Tirana     Lortzing  1801  1851  de  m  Ali Pascha von Janina   
1    1213  al   Tirana       Mozart  1756  1791  at  m           Don Giovanni   
2    1213  al   Tirana      Puccini  1858  1924  it  m                  Tosca   
3    1213  am  Yerevan  Spendiaryan  1871  1928  am  m                 Almast   
4    1213  am  Yerevan    Tigranian  1879  1950  am  m                 Anoush   

  worknat  ... perf_total_per_10k_co_pop Season Year  \
0      de  ...                  0.385301  2013-01-01   
1      it  ...                  0.385301  2013-01-01   
2      it  ...                  0.385301  2013-01-01   
3      am  ...                  0.371147  2013-01-01   
4      am  ...                  0.371147  2013-01-01   

   country_change_from_previous_season city_change_from_previous_season  Year  \
0                             2.230995                         2.230995  2013   
1             

In [39]:
nan_count_per_column = opera.isna().sum()
print(nan_count_per_column)

season                                     0
iso                                        0
city                                       0
composer                                   5
db                                       227
dd                                      3593
nat                                       11
mf                                         0
work                                       5
worknat                                 1548
type                                       0
start date                                 0
performances                               0
Country Name                               0
city population                            0
country population                         0
continent                                  0
sub-region                                 0
performances_season_by_country             0
perf_per_10k_ppl_co_pop                    0
performances_season_by_city                0
perf_per_1k_ppl_city_pop                   0
opera_by_c

In [41]:
from sklearn.metrics import classification_report, confusion_matrix,recall_score , accuracy_score, precision_score, roc_auc_score

#Evaluate the model using accuracy, precision, and recall scores under evaluate_model fuction
def evaluate_model(y_test, y_pred):
    print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
    print(f"Precision (macro): {precision_score(y_test, y_pred, average='macro')}")
    print(f"Recall (macro): {recall_score(y_test, y_pred, average='macro')}")

In [43]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

#Evaluate the model using mean absolute error, mean standard error, root mean square error and the rsquared score under lin_evaluate_model fuction
def lin_evaluate_model(y_test, y_pred):
    print(f"MAE: {mean_absolute_error(y_test, y_pred)}")
    print(f"MSE: {mean_squared_error(y_test, y_pred)}")
    print(f"RMSE: {mean_squared_error(y_test, y_pred, squared=False)}")
    print(f"R2_Score: {r2_score(y_test, y_pred)}")

In [45]:
final_results = []

Linear Regression models - I ran these first without grouping by Country to help the predictive model just as a starting point to see how a linear regression model would perform

In [48]:
X_country = opera[['country population', 'performances_season_by_country', 'perf_per_10k_ppl_co_pop',
                   'city population', 'performances_season_by_city', 'perf_per_1k_ppl_city_pop', 'Year', 'Month', 'Day', 
                   'Weekday', 'Week_of_Year', 'Days_Since_Start']]

# Dependent variable (growth at country level)
y_country = opera['country_change_from_previous_season']

# Train-test split
X_train, X_test, y_train_country, y_test_country = train_test_split(X_country, y_country, test_size=0.2, random_state=42)

# Initialize the StandardScaler
scaler = StandardScaler()

# Fit and transform the training data
X_train_scaled = scaler.fit_transform(X_train)

# Transform the test data (using the same scaler as the training data)
X_test_scaled = scaler.transform(X_test)

# Model for country growth prediction
lin_country = LinearRegression()

# Train the model on scaled data
lin_country.fit(X_train_scaled, y_train_country)

# Transform the entire country data to scaled values before prediction (using the same scaler)
X_country_scaled = scaler.transform(X_country)

# Predict growth for the entire country dataset
opera['predicted_country_growth'] = lin_country.predict(X_country_scaled)

# Sort countries based on predicted growth
df_country_growth_sorted = opera[['Country Name', 'predicted_country_growth']].sort_values(by='predicted_country_growth', ascending=False)

# Drop duplicates and keep the row with the highest growth for each city
df_opera_country_growth_unique = df_country_growth_sorted.drop_duplicates(subset='Country Name', keep='first')

# Get top 10 countries with highest predicted growth
top_countries_predicted = df_opera_country_growth_unique.head(10)
print(top_countries_predicted)

      Country Name  predicted_country_growth
263      Australia                 75.180739
190        Austria                 72.237065
2427       Finland                 63.241543
3883       Romania                 59.168818
4515      Slovenia                 58.507863
16338      Hungary                 58.266885
3390     Lithuania                 58.138222
3798        Poland                 56.789262
4615        Turkey                 47.124328
14132  Switzerland                 46.597059


In [50]:
# Make a prediction
y_pred_country_lin = lin_country.predict(X_test_scaled)

# Evaluate model
lin_evaluate_model(y_test_country, y_pred_country_lin)

MAE: 147.3972739647199
MSE: 41254.4963747648
RMSE: 203.1120291237444
R2_Score: 0.038655174243710566


In [52]:
# Independent variables (predictors)
X_city = opera[['country population', 'performances_season_by_country', 'perf_per_10k_ppl_co_pop',
                   'city population', 'performances_season_by_city', 'perf_per_1k_ppl_city_pop', 'Year', 'Month', 'Day', 
                   'Weekday', 'Week_of_Year', 'Days_Since_Start']]

# Dependent variable (growth at city level)
y_city = opera['city_change_from_previous_season']

# Train-test split
X_train, X_test, y_train_city, y_test_city = train_test_split(X_city, y_city, test_size=0.2, random_state=42)

# Initialize the StandardScaler
scaler = StandardScaler()

# Fit and transform the training data
X_train_scaled = scaler.fit_transform(X_train)

# Transform the test data (using the same scaler as the training data)
X_test_scaled = scaler.transform(X_test)

# Model for city growth prediction
lin_city = LinearRegression()

# Train the model on scaled data
lin_city.fit(X_train_scaled, y_train_city)

# Transform the entire city data to scaled values before prediction (using the same scaler)
X_city_scaled = scaler.transform(X_city)

# Predict growth for the entire city dataset
opera['predicted_city_growth'] = lin_country.predict(X_city_scaled)

# Sort cities based on predicted growth
df_city_growth_sorted = opera[['iso', 'city', 'predicted_city_growth']].sort_values(by='predicted_city_growth', ascending=False)

# Drop duplicates and keep the row with the highest growth for each city
df_opera_city_growth_unique = df_city_growth_sorted.drop_duplicates(subset='city', keep='first')

# Get top 25 cities with highest predicted growth
top_cities_predicted = df_opera_city_growth_unique.head(10)
print(top_cities_predicted)

      iso         city  predicted_city_growth
263    au       Sydney              75.180739
190    at       Vienna              72.237065
2427   fi     Helsinki              63.241543
3883   ro  Cluj-Napoca              59.168818
4515   si      Maribor              58.507863
16338  hu     Budapest              58.266885
3390   lt      Vilnius              58.138222
3798   pl      Wroclaw              56.789262
10736  ro    Timisoara              55.113396
10113  lt       Kaunas              53.492196


In [54]:
# Make a prediction
y_pred_city_lin = lin_city.predict(X_test_scaled)

# Evaluate model
lin_evaluate_model(y_test_city, y_pred_city_lin)

MAE: 20.428280777427656
MSE: 5380.81251451332
RMSE: 73.35402180189796
R2_Score: 0.0036593842846965874


Linear Regression and grouping by sub-region for predictive analysis. This time I grouped the opera data by sub-regions before peforming the linear regression to see if it improved predictions.  The scores were significantly better for the cities prediction but they still were underperforming for the country data. 

In [98]:
# Independent variables (predictors)
X_country = opera[['country population', 'performances_season_by_country', 'perf_per_10k_ppl_co_pop',
                   'city population', 'performances_season_by_city', 'perf_per_1k_ppl_city_pop', 'Year', 'Month', 'Day', 
                   'Weekday', 'Week_of_Year', 'Days_Since_Start']]

# Dependent variable (growth at country level)
y_country = opera['country_change_from_previous_season']

# Group by Country
country_groups = opera.groupby('sub-region')

# Initialize the StandardScaler
scaler = StandardScaler()

# Model for country growth prediction
lin_country = LinearRegression()

# Store the predicted values in a DataFrame
predicted_growth_values = pd.Series(index=opera.index)  # Create an empty Series to store predictions

# Loop through each sub-region group to train and predict separately
for country_name, group in country_groups:
    # If the group has more than one sample, split and train
    if len(group) > 1:
        # Extract the features and target for this country group
        X_group = group[['country population', 'performances_season_by_country', 'perf_per_10k_ppl_co_pop', 
                         'city population', 'performances_season_by_city', 'perf_per_1k_ppl_city_pop', 'Year', 'Month', 'Day', 
                         'Weekday', 'Week_of_Year', 'Days_Since_Start']]
        y_group = group['country_change_from_previous_season']

        # Split data into train and test sets (80% train, 20% test)
        X_train, X_test, y_train_country, y_test_country = train_test_split(X_group, y_group, test_size=0.2, random_state=42)

        # Scale the training data
        X_train_scaled = scaler.fit_transform(X_train)

        # Train the model on the training data
        lin_country.fit(X_train_scaled, y_train_country)

        # Scale the test data using the same scaler
        X_test_scaled = scaler.transform(X_test)

        # Make predictions for the test set
        predictions = lin_country.predict(X_test_scaled)

        # Assign predictions to the correct indices in the original DataFrame
        predicted_growth_values.loc[X_test.index] = predictions  # Correctly assign predictions to test set rows
    else:
        # If the group has only one sample, predict directly and append
        X_group = group[['country population', 'performances_season_by_country', 'perf_per_10k_ppl_co_pop', 
                         'city population', 'performances_season_by_city', 'perf_per_1k_ppl_city_pop', 'Year', 'Month', 'Day', 
                         'Weekday', 'Week_of_Year', 'Days_Since_Start']]
        y_group = group['country_change_from_previous_season']
        
        # Scale the data for this group
        X_scaled = scaler.fit_transform(X_group)

        # Train the model
        lin_country.fit(X_scaled, y_group)

        # Make predictions for this group
        predictions = lin_country.predict(X_scaled)
        
        # Assign predictions to the correct indices in the original DataFrame
        predicted_growth_values.loc[group.index] = predictions  # Correctly assign predictions to the original rows

# Assign the predicted values to the original DataFrame
opera['predicted_country_growth'] = predicted_growth_values

# Sort countries based on predicted growth
df_country_growth_sorted = opera[['Country Name', 'predicted_country_growth']].sort_values(by='predicted_country_growth', ascending=False)

# Drop duplicates and keep the row with the highest growth for each country
df_opera_country_growth_unique = df_country_growth_sorted.drop_duplicates(subset='Country Name', keep='first')

# Get top 10 countries with highest predicted growth
top_countries_predicted = df_opera_country_growth_unique.head(10)
print(top_countries_predicted)

             Country Name  predicted_country_growth
12366       United States                384.528795
16879          Kazakhstan                353.238890
10028          Kyrgyzstan                325.570095
32851          Uzbekistan                270.377483
30634  Russian Federation                259.203609
33363              Canada                248.547297
17247              Poland                180.091249
3100                Italy                 92.900284
15701               Egypt                 90.607635
3917              Romania                 77.953992


In [100]:
# Make a prediction
y_pred_country_lin = lin_country.predict(X_test_scaled)

# Evaluate model
lin_evaluate_model(y_test_country, y_pred_country_lin)

MAE: 132.23534786689217
MSE: 28676.095731319605
RMSE: 169.34017754602598
R2_Score: 0.042523402390002074


In [102]:
# Independent variables (predictors)
X_city = opera[['country population', 'performances_season_by_country', 'perf_per_10k_ppl_co_pop',
                   'city population', 'performances_season_by_city', 'perf_per_1k_ppl_city_pop', 'Year', 'Month', 'Day', 
                   'Weekday', 'Week_of_Year', 'Days_Since_Start']]

# Dependent variable (growth at city level)
y_city = opera['city_change_from_previous_season']

# Still group the cities by Country
city_groups = opera.groupby('sub-region')

# Initialize the StandardScaler
scaler = StandardScaler()

# Model for country growth prediction
lin_city = LinearRegression()

# Store the predicted values in a DataFrame
predicted_growth_values = pd.Series(index=opera.index)  # Create an empty Series to store predictions

# Loop through each country group to train and predict separately
for city_name, group in city_groups:
    # If the group has more than one sample, split and train
    if len(group) > 1:
        # Extract the features and target for this country group
        X_group = group[['country population', 'performances_season_by_country', 'perf_per_10k_ppl_co_pop', 
                         'city population', 'performances_season_by_city', 'perf_per_1k_ppl_city_pop', 'Year', 'Month', 'Day', 
                         'Weekday', 'Week_of_Year', 'Days_Since_Start']]
        y_group = group['city_change_from_previous_season']

        # Split data into train and test sets (80% train, 20% test)
        X_train, X_test, y_train_city, y_test_city = train_test_split(X_group, y_group, test_size=0.2, random_state=42)

        # Scale the training data
        X_train_scaled = scaler.fit_transform(X_train)

        # Train the model on the training data
        lin_city.fit(X_train_scaled, y_train_city)

        # Scale the test data using the same scaler
        X_test_scaled = scaler.transform(X_test)

        # Make predictions for the test set
        predictions = lin_city.predict(X_test_scaled)

        # Assign predictions to the correct indices in the original DataFrame
        predicted_growth_values.loc[X_test.index] = predictions  # Correctly assign predictions to test set rows
    else:
        # If the group has only one sample, predict directly and append
        X_group = group[['country population', 'performances_season_by_country', 'perf_per_10k_ppl_co_pop', 
                         'city population', 'performances_season_by_city', 'perf_per_1k_ppl_city_pop', 'Year', 'Month', 'Day', 
                         'Weekday', 'Week_of_Year', 'Days_Since_Start']]
        y_group = group['city_change_from_previous_season']
        
        # Scale the data for this group
        X_scaled = scaler.fit_transform(X_group)

        # Train the model
        lin_city.fit(X_scaled, y_group)

        # Make predictions for this group
        predictions = lin_city.predict(X_scaled)
        
        # Assign predictions to the correct indices in the original DataFrame
        predicted_growth_values.loc[group.index] = predictions  # Correctly assign predictions to the original rows

# Assign the predicted values to the original DataFrame
opera['predicted_city_growth'] = predicted_growth_values

# Sort countries based on predicted growth
df_city_growth_sorted = opera[['iso', 'city', 'predicted_city_growth']].sort_values(by='predicted_city_growth', ascending=False)

# Drop duplicates and keep the row with the highest growth for each country
df_opera_city_growth_unique = df_city_growth_sorted.drop_duplicates(subset='city', keep='first')

# Get top 10 countries with highest predicted growth
top_cities_predicted = df_opera_city_growth_unique.head(10)
print(top_cities_predicted)

      iso           city  predicted_city_growth
15701  eg     Alexandria              71.954844
30467  ru         Moscow              52.171187
26239  us     Washington              51.766670
10389  nz     Berhampore              42.547934
13054  us    Saint Louis              41.165920
11188  ru  St Petersburg              38.523518
11980  us      Arlington              34.697543
6536   am        Yerevan              34.630684
12661  us        Madison              32.847327
12412  us        Houston              32.374409


In [104]:
# Make a prediction
y_pred_city_lin = lin_city.predict(X_test_scaled)

# Evaluate model
lin_evaluate_model(y_test_city, y_pred_city_lin)

MAE: 16.597406069575385
MSE: 502.9302155600541
RMSE: 22.426105670848294
R2_Score: 0.011436314558350591


Logistic Regression Models - let's see if we can get a better predictive model using Logistic regression without countries being grouped first and then try it with grouping countries. In order to run a Logistic regression model, I binned the country_change_from_previous_season by quartiles into categories of Low, Medium, High and Very High so there is a category that is being used as the dependent outcome.

In [66]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, f1_score

# Independent variables (predictors)
X_country = opera[['country population', 'performances_season_by_country', 'perf_per_10k_ppl_co_pop',
                   'city population', 'performances_season_by_city', 'perf_per_1k_ppl_city_pop', 'Year', 'Month', 'Day', 
                   'Weekday', 'Week_of_Year', 'Days_Since_Start']]

# Dependent variable (growth at country level)
y_country = opera['country_change_from_previous_season']

# Convert continuous growth to categorical labels (Low, Medium, High)
q1 = opera['country_change_from_previous_season'].quantile(0.25)
q2 = opera['country_change_from_previous_season'].quantile(0.50)
q3 = opera['country_change_from_previous_season'].quantile(0.75)

bins = [-float('inf'), q1, q2, q3, float('inf')]
labels = ['Low', 'Medium', 'High', 'Very High']

y_country_classified = pd.cut(y_country, bins=bins, labels=labels)

# Train-test split
X_train, X_test, y_train_country, y_test_country = train_test_split(X_country, y_country_classified, test_size=0.2, random_state=42)

# Initialize the StandardScaler
scaler = StandardScaler()

# Fit and transform the training data
X_train_scaled = scaler.fit_transform(X_train)

# Transform the test data (using the same scaler as the training data)
X_test_scaled = scaler.transform(X_test)

In [68]:
# Initialize LogisticRegression
lr_country = LogisticRegression(max_iter=10000)

# Define the hyperparameter grid for tuning
param_grid = {
    'C': [0.1, 1, 10],  # Regularization strength
    'solver': ['liblinear', 'saga'],  # Use 'liblinear' and 'saga' or 'lbfgs' for L1
}

# Define the custom scoring function (e.g., F1 score)
custom_scorer = make_scorer(f1_score, average='weighted')

# GridSearchCV with custom scoring
grid_search = GridSearchCV(estimator=lr_country, param_grid=param_grid, scoring=custom_scorer, cv=5)
grid_search.fit(X_train_scaled, y_train_country)

# Output the best parameters and the best F1 score
print("Best Parameters:", grid_search.best_params_)
print("Best F1 Score:", grid_search.best_score_)

Best Parameters: {'C': 0.1, 'solver': 'saga'}
Best F1 Score: 0.4923619075830346


In [70]:
# Best parameters found during the grid search
best_params = grid_search.best_params_

# Create the Logistic Regression model with the best parameters
lr_country = LogisticRegression(
    C=best_params['C'],
    solver=best_params['solver'],
    max_iter=10000
)

# Train the model on scaled data
lr_country.fit(X_train_scaled, y_train_country)

# Transform the entire country data to scaled values before prediction (using the same scaler)
X_country_scaled = scaler.transform(X_country)

# Predict growth for the entire country dataset
opera['predicted_country_growth'] = lr_country.predict(X_country_scaled)

# Sort countries based on predicted growth
df_country_growth_sorted = opera[['Country Name', 'predicted_country_growth']].sort_values(by='predicted_country_growth', ascending=False)

# Drop duplicates and keep the row with the highest growth for each country
df_opera_country_growth_unique = df_country_growth_sorted.drop_duplicates(subset='Country Name', keep='first')

# Get top 10 countries with highest predicted growth
top_countries_predicted = df_opera_country_growth_unique.head(10)
print(top_countries_predicted)

             Country Name predicted_country_growth
14746             Germany                Very High
4408   Russian Federation                Very High
16534               Italy                Very High
19042       United States                Very High
35292              France                Very High
11794      United Kingdom                   Medium
10536              Poland                   Medium
13624             Austria                   Medium
10652             Romania                   Medium
20573         Switzerland                   Medium


In [72]:
#Make a prediction
y_pred_country_lr = lr_country.predict(X_test_scaled)
accuracy = accuracy_score(y_test_country, y_pred_country_lr)
final_results.append(accuracy)
evaluate_model(y_test_country, y_pred_country_lr)

Accuracy: 0.5217000513610683
Precision (macro): 0.4954212985332075
Recall (macro): 0.519426396833789


In [106]:
# Independent variables (predictors)
X_country = opera[['country population', 'performances_season_by_country', 'perf_per_10k_ppl_co_pop',
                   'city population', 'performances_season_by_city', 'perf_per_1k_ppl_city_pop', 'Year', 'Month', 'Day', 
                   'Weekday', 'Week_of_Year', 'Days_Since_Start']]

# Dependent variable (growth at country level)
y_country = opera['country_change_from_previous_season']

# Convert continuous growth to categorical labels (Low, Medium, High)
q1 = opera['country_change_from_previous_season'].quantile(0.25)
q2 = opera['country_change_from_previous_season'].quantile(0.50)
q3 = opera['country_change_from_previous_season'].quantile(0.75)

bins = [-float('inf'), q1, q2, q3, float('inf')]
labels = ['Low', 'Medium', 'High', 'Very High']

y_country_classified = pd.cut(y_country, bins=bins, labels=labels)

# Train-test split
X_train, X_test, y_train_country, y_test_country = train_test_split(X_country, y_country_classified, test_size=0.2, random_state=42)

# Initialize the StandardScaler
scaler = StandardScaler()

# Fit and transform the training data
X_train_scaled = scaler.fit_transform(X_train)

# Transform the test data (using the same scaler as the training data)
X_test_scaled = scaler.transform(X_test)

In [108]:
# Initialize LogisticRegression
lr_country = LogisticRegression(max_iter=10000, multi_class='ovr')

# Define the hyperparameter grid for tuning
param_grid = {
    'C': [0.1, 1, 10],  # Regularization strength
    'solver': ['liblinear', 'saga'],  # Use 'liblinear' and 'saga' or 'lbfgs' for L1
}

# Define the custom scoring function (e.g., F1 score)
custom_scorer = make_scorer(f1_score, average='weighted')

# GridSearchCV with custom scoring
grid_search = GridSearchCV(estimator=lr_country, param_grid=param_grid, scoring=custom_scorer, cv=5)
grid_search.fit(X_train_scaled, y_train_country)

# Output the best parameters and the best F1 score
print("Best Parameters:", grid_search.best_params_)
print("Best F1 Score:", grid_search.best_score_)

Best Parameters: {'C': 10, 'solver': 'saga'}
Best F1 Score: 0.48709776078392775


In [109]:
# Best parameters found during the grid search
best_params = grid_search.best_params_

# Create the Logistic Regression model with the best parameters
lr_country = LogisticRegression(
    C=best_params['C'],
    solver=best_params['solver'],
    max_iter=10000,
    multi_class='ovr'  # Specify multiclass handling (One-vs-Rest)
)

# Initialize a container to store predictions with correct dtype (categorical or object)
predicted_growth_values = pd.Series(index=opera.index, dtype='object')

# Group the data by 'Country Name'
country_groups = opera.groupby('sub-region')

# Loop through each country group to train and predict separately
for country_name, group in country_groups:
    # If the group has more than one sample and more than one class, split and train
    if len(group) > 1 and len(group['country_change_from_previous_season'].unique()) > 1:
        # Extract the features and target for this country group
        X_group = group[['country population', 'performances_season_by_country', 'perf_per_10k_ppl_co_pop', 
                         'city population', 'performances_season_by_city', 'perf_per_1k_ppl_city_pop', 'Year', 'Month', 'Day', 
                         'Weekday', 'Week_of_Year', 'Days_Since_Start']]
        y_group = group['country_change_from_previous_season']

        # Convert the target to categorical labels (same as before)
        y_group_classified = pd.cut(y_group, bins=bins, labels=labels)

        # Split data into train and test sets (80% train, 20% test)
        X_train, X_test, y_train_country, y_test_country = train_test_split(X_group, y_group_classified, test_size=0.2, random_state=42)

        # Scale the training data
        X_train_scaled = scaler.fit_transform(X_train)

        # **Only train if the target variable has more than one class**
        if len(y_train_country.unique()) > 1:
            # Train the model on the training data
            lr_country.fit(X_train_scaled, y_train_country)

            # Scale the test data using the same scaler
            X_test_scaled = scaler.transform(X_test)

            # Make predictions for the test set
            predictions = lr_country.predict(X_test_scaled)

            # Assign predictions to the correct indices in the original DataFrame
            predicted_growth_values.loc[X_test.index] = predictions  # Correctly assign predictions to test set rows
        else:
            print(f"Skipping country: {country_name} due to insufficient class variation in the target.")
            continue
    else:
        # Skip the group if it has only one class or one sample
        print(f"Skipping country: {country_name} due to insufficient class variation or sample size.")
        continue

# You can now inspect or save the results
opera['predicted_country_growth'] = predicted_growth_values

# Sort and get the top countries based on predicted growth
df_country_growth_sorted = opera[['Country Name', 'predicted_country_growth']].sort_values(by='predicted_country_growth', ascending=False)

# Drop duplicates and keep the row with the highest growth for each country
df_opera_country_growth_unique = df_country_growth_sorted.drop_duplicates(subset='Country Name', keep='first')

# Get top 10 countries with highest predicted growth
top_countries_predicted = df_opera_country_growth_unique.head(10)
print(top_countries_predicted)

/var/folders/v8/j1kjtws546j8drwljwyklgrm0000gn/T/ipykernel_6184/1954064732.py:49: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['Medium' 'High' 'High' 'High' 'High' 'High' 'High' 'High' 'Medium' 'High'
 'High' 'Medium' 'High' 'High' 'High' 'Medium' 'High' 'High' 'High' 'High'
 'High' 'High' 'High' 'High' 'High' 'High' 'High' 'High' 'Medium' 'High'
 'High' 'High' 'High' 'High' 'High' 'Medium' 'High' 'High' 'High' 'High'
 'High' 'High' 'High' 'High' 'High' 'High' 'High' 'High' 'High' 'Medium'
 'High' 'High' 'High' 'High' 'High' 'High' 'High' 'Medium' 'High' 'High'
 'High' 'High' 'Medium' 'High']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  predicted_growth_values.loc[X_test.index] = predictions  # Correctly assign predictions to test set rows


Skipping country: Northern Africa due to insufficient class variation in the target.
Skipping country: South-eastern Asia due to insufficient class variation in the target.
             Country Name predicted_country_growth
14846             Germany                Very High
13313       United States                Very High
35890               Italy                Very High
30564  Russian Federation                Very High
11596      United Kingdom                Very High
9024                Spain                Very High
18084              Sweden                Very High
35573             Hungary                   Medium
27086      Czech Republic                   Medium
15895              France                   Medium


In [110]:
#Make a prediction
y_pred_country_lr = lr_country.predict(X_test_scaled)
accuracy = accuracy_score(y_test_country, y_pred_country_lr)
final_results.append(accuracy)
evaluate_model(y_test_country, y_pred_country_lr)

Accuracy: 0.6017592962814874
Precision (macro): 0.6043315110531736
Recall (macro): 0.6110848694378016


In [114]:
# Independent variables (predictors)
X_city = opera[['country population', 'performances_season_by_country', 'perf_per_10k_ppl_co_pop',
                   'city population', 'performances_season_by_city', 'perf_per_1k_ppl_city_pop', 'Year', 'Month', 'Day', 
                   'Weekday', 'Week_of_Year', 'Days_Since_Start']]

# Dependent variable (growth at city level)
y_city = opera['city_change_from_previous_season']

# Convert continuous growth to categorical labels (Low, Medium, High)
q1 = opera['city_change_from_previous_season'].quantile(0.25)
q2 = opera['city_change_from_previous_season'].quantile(0.50)
q3 = opera['city_change_from_previous_season'].quantile(0.75)

bins = [-float('inf'), q1, q2, q3, float('inf')]
labels = ['Low', 'Medium', 'High', 'Very High']

y_city_classified = pd.cut(y_city, bins=bins, labels=labels)

# Train-test split
X_train, X_test, y_train_city, y_test_city = train_test_split(X_city, y_city_classified, test_size=0.2, random_state=42)

# Initialize the StandardScaler
scaler = StandardScaler()

# Fit and transform the training data
X_train_scaled = scaler.fit_transform(X_train)

# Transform the test data (using the same scaler as the training data)
X_test_scaled = scaler.transform(X_test)

In [116]:
# Initialize LogisticRegression
lr_city = LogisticRegression(max_iter=10000)

# Define the hyperparameter grid for tuning
param_grid = {
    'C': [0.1, 1, 10],  # Regularization strength
    'solver': ['liblinear', 'saga'],  # Use 'liblinear' and 'saga' or 'lbfgs' for L1
}

# Define the custom scoring function (e.g., F1 score)
custom_scorer = make_scorer(f1_score, average='weighted')

# GridSearchCV with custom scoring
grid_search = GridSearchCV(estimator=lr_city, param_grid=param_grid, scoring=custom_scorer, cv=5)
grid_search.fit(X_train_scaled, y_train_city)

# Output the best parameters and the best F1 score
print("Best Parameters:", grid_search.best_params_)
print("Best F1 Score:", grid_search.best_score_)

Best Parameters: {'C': 0.1, 'solver': 'saga'}
Best F1 Score: 0.32796112406192546


In [117]:
# Best parameters found during the grid search
best_params = grid_search.best_params_

# Create the Logistic Regression model with the best parameters
lr_city = LogisticRegression(
    C=best_params['C'],
    solver=best_params['solver'],
    max_iter=10000
)

# Train the model on scaled data
lr_city.fit(X_train_scaled, y_train_city)

# Transform the entire city data to scaled values before prediction (using the same scaler)
X_city_scaled = scaler.transform(X_city)

# Predict growth for the entire city dataset
opera['predicted_city_growth'] = lr_city.predict(X_city_scaled)

# Sort countries based on predicted growth
df_city_growth_sorted = opera[['iso', 'city', 'predicted_city_growth']].sort_values(by='predicted_city_growth', ascending=False)

# Drop duplicates and keep the row with the highest growth for each city
df_opera_city_growth_unique = df_city_growth_sorted.drop_duplicates(subset='city', keep='first')

# Get top 10 cities with highest predicted growth
top_cities_predicted = df_opera_city_growth_unique.head(10)
print(top_cities_predicted)

      iso          city predicted_city_growth
19469  us      New York             Very High
31379  uk        London             Very High
14771  de     Frankfurt             Very High
14754  de         Essen             Very High
31548  us     Arlington             Very High
14879  de         Hagen             Very High
14914  de       Hamburg             Very High
14472  de        Berlin             Very High
14528  de  Braunschweig             Very High
14534  de        Bremen             Very High


In [118]:
#Make a prediction
y_pred_city_lr = lr_city.predict(X_test_scaled)
accuracy = accuracy_score(y_test_city, y_pred_city_lr)
final_results.append(accuracy)
evaluate_model(y_test_city, y_pred_city_lr)

Accuracy: 0.34232152028762197
Precision (macro): 0.3622382334573313
Recall (macro): 0.33420688494479084


In [122]:
# Independent variables (predictors)
X_city = opera[['country population', 'performances_season_by_country', 'perf_per_10k_ppl_co_pop',
                   'city population', 'performances_season_by_city', 'perf_per_1k_ppl_city_pop', 'Year', 'Month', 'Day', 
                   'Weekday', 'Week_of_Year', 'Days_Since_Start']]

# Dependent variable (growth at city level)
y_city = opera['city_change_from_previous_season']

# Convert continuous growth to categorical labels (Low, Medium, High)
q1 = opera['city_change_from_previous_season'].quantile(0.25)
q2 = opera['city_change_from_previous_season'].quantile(0.50)
q3 = opera['city_change_from_previous_season'].quantile(0.75)

bins = [-float('inf'), q1, q2, q3, float('inf')]
labels = ['Low', 'Medium', 'High', 'Very High']

y_city_classified = pd.cut(y_city, bins=bins, labels=labels)

# Train-test split
X_train, X_test, y_train_city, y_test_city = train_test_split(X_city, y_city_classified, test_size=0.2, random_state=42)

# Initialize the StandardScaler
scaler = StandardScaler()

# Fit and transform the training data
X_train_scaled = scaler.fit_transform(X_train)

# Transform the test data (using the same scaler as the training data)
X_test_scaled = scaler.transform(X_test)

In [124]:
# Initialize LogisticRegression
lr_city = LogisticRegression(max_iter=10000)

# Define the hyperparameter grid for tuning
param_grid = {
    'C': [0.1, 1, 10],  # Regularization strength
    'solver': ['liblinear', 'saga'],  # Use 'liblinear' and 'saga' or 'lbfgs' for L1
}

# Define the custom scoring function (e.g., F1 score)
custom_scorer = make_scorer(f1_score, average='weighted')

# GridSearchCV with custom scoring
grid_search = GridSearchCV(estimator=lr_city, param_grid=param_grid, scoring=custom_scorer, cv=5)
grid_search.fit(X_train_scaled, y_train_city)

# Output the best parameters and the best F1 score
print("Best Parameters:", grid_search.best_params_)
print("Best F1 Score:", grid_search.best_score_)

Best Parameters: {'C': 10, 'solver': 'liblinear'}
Best F1 Score: 0.3248013022646522


In [130]:
# Best parameters found during the grid search
best_params = grid_search.best_params_

# Create the Logistic Regression model with the best parameters
lr_city = LogisticRegression(
    C=best_params['C'],
    solver=best_params['solver'],
    max_iter=10000,
    multi_class='ovr'  # Specify multiclass handling (One-vs-Rest)
)

# Initialize a container to store predictions with correct dtype (categorical or object)
predicted_growth_values = pd.Series(index=opera.index, dtype='object')

# Still group the data by 'sub-region'
city_groups = opera.groupby('sub-region')

# Loop through each sub-region group to train and predict separately for cities
for city_name, group in city_groups:
    # If the group has more than one sample and more than one class, split and train
    if len(group) > 1 and len(group['city_change_from_previous_season'].unique()) > 1:
        # Extract the features and target for this sub-region group
        X_group = group[['country population', 'performances_season_by_country', 'perf_per_10k_ppl_co_pop', 
                         'city population', 'performances_season_by_city', 'perf_per_1k_ppl_city_pop', 'Year', 'Month', 'Day', 
                         'Weekday', 'Week_of_Year', 'Days_Since_Start']]
        y_group = group['city_change_from_previous_season']

        # Convert the target to categorical labels (same as before)
        y_group_classified = pd.cut(y_group, bins=bins, labels=labels)

        # Split data into train and test sets (80% train, 20% test)
        X_train, X_test, y_train_city, y_test_city = train_test_split(X_group, y_group_classified, test_size=0.2, random_state=42)

        # Scale the training data
        X_train_scaled = scaler.fit_transform(X_train)

        # **Only train if the target variable has more than one class**
        if len(y_train_city.unique()) > 1:
            # Train the model on the training data
            lr_city.fit(X_train_scaled, y_train_city)

            # Scale the test data using the same scaler
            X_test_scaled = scaler.transform(X_test)

            # Make predictions for the test set
            predictions = lr_city.predict(X_test_scaled)

            # Ensure the indices of predictions and X_test match
            predicted_growth_values.loc[X_test.index] = predictions  # Correctly assign predictions to test set rows
        else:
            print(f"Skipping sub-region: {city_name} due to insufficient class variation in the target.")
            continue
    else:
        # Skip the group if it has only one class or one sample
        print(f"Skipping sub-region: {city_name} due to insufficient class variation or sample size.")
        continue

# You can now inspect or save the results
opera['predicted_city_growth'] = predicted_growth_values

# Sort and get the top cities based on predicted growth
df_city_growth_sorted = opera[['iso', 'city', 'predicted_city_growth']].sort_values(by='predicted_city_growth', ascending=False)

# Drop duplicates and keep the row with the highest growth for each city
df_opera_city_growth_unique = df_city_growth_sorted.drop_duplicates(subset='city', keep='first')

# Get top 10 cities with highest predicted growth
top_cities_predicted = df_opera_city_growth_unique.head(10)
print(top_cities_predicted)

      iso          city predicted_city_growth
38935  uz      Tashkent             Very High
32944  at          Graz             Very High
33004  at      Salzburg             Very High
32984  at          Linz             Very High
32959  at     Innsbruck             Very High
13343  us    Washington             Very High
32898  ar  Buenos Aires             Very High
32868  za     Cape Town             Very High
32839  us    Wilmington             Very High
33126  au     Melbourne             Very High


In [132]:
#Make a prediction
y_pred_city_lr = lr_city.predict(X_test_scaled)
accuracy = accuracy_score(y_test_city, y_pred_city_lr)
final_results.append(accuracy)
evaluate_model(y_test_city, y_pred_city_lr)

Accuracy: 0.3790483806477409
Precision (macro): 0.4056179485771411
Recall (macro): 0.37454003271170466


Random Forest Classifier since the models for countries and cities were not very strong with logistic regression and might work better with RCF since it's a stronger model.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd

# Independent variables (predictors)
X_country = opera[['country population', 'performances_season_by_country', 'perf_per_10k_ppl_co_pop',
                 'city population', 'performances_season_by_city', 'perf_per_1k_ppl_city_pop', 
                 'Year', 'Month', 'Day', 'Weekday', 'Week_of_Year', 'Days_Since_Start']]

# Dependent variable (growth at city level)
y_country = opera['city_change_from_previous_season']

# Convert continuous growth to categorical labels (Low, Medium, High)
q1 = opera['city_change_from_previous_season'].quantile(0.25)
q2 = opera['city_change_from_previous_season'].quantile(0.50)
q3 = opera['city_change_from_previous_season'].quantile(0.75)

bins = [-float('inf'), q1, q2, q3, float('inf')]
labels = ['Low', 'Medium', 'High', 'Very High']

y_city_classified = pd.cut(y_city, bins=bins, labels=labels)

# Train-test split (80% train, 20% test)
X_train, X_test, y_train_city, y_test_city = train_test_split(X_city, y_city_classified, test_size=0.2, random_state=42)

# Initialize the StandardScaler
scaler = StandardScaler()

# Fit and transform the training data
X_train_scaled = scaler.fit_transform(X_train)

# Transform the test data (using the same scaler as the training data)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Hyperparameter grid for GridSearchCV
param_dist = {
    'n_estimators': np.arange(100, 1001, 100),  # 100, 200, ..., 1000
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2'],  # Added this line
    'bootstrap': [True, False]
}

# Initialize RandomForestClassifier
rf_city = RandomForestClassifier(random_state=42)

# Define a custom scoring function (e.g., F1 score)
custom_scorer = make_scorer(f1_score, average='weighted')

# GridSearchCV initialization with custom scoring
grid_search = GridSearchCV(estimator=rf_city, 
                           param_grid=param_dist, 
                           cv=5,  # Number of folds in cross-validation
                           n_jobs=-1,  # Use all CPU cores
                           verbose=2,  # Print progress messages
                           scoring=custom_scorer)  # Use F1 score for evaluation

# Fit the model
grid_search.fit(X_train_scaled, y_train_city)

# Output the best parameters and the best F1 score
print("Best Parameters:", grid_search.best_params_)
print("Best F1 Score:", grid_search.best_score_)

In [ ]:
# Initialize a container to store predictions with correct dtype (categorical or object)
predicted_growth_values = pd.Series(index=opera.index, dtype='object')

# Still group the data by 'sub-region'
city_groups = opera.groupby('sub-region')

# Loop through each country group to train and predict separately for cities
for city_name, group in city_groups:
    # If the group has more than one sample and more than one class, split and train
    if len(group) > 1 and len(group['city_change_from_previous_season'].unique()) > 1:
        # Extract the features and target for this country group
        X_group = group[['country population', 'performances_season_by_country', 'perf_per_10k_ppl_co_pop', 
                         'city population', 'performances_season_by_city', 'perf_per_1k_ppl_city_pop', 'Year', 'Month', 'Day', 
                         'Weekday', 'Week_of_Year', 'Days_Since_Start']]
        y_group = group['city_change_from_previous_season']

        # Convert the target to categorical labels (same as before)
        y_group_classified = pd.cut(y_group, bins=bins, labels=labels)

        # Split data into train and test sets (80% train, 20% test)
        X_train, X_test, y_train_city, y_test_city = train_test_split(X_group, y_group_classified, test_size=0.2, random_state=42)

        # Scale the training data
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)

        # **Only train if the target variable has more than one class**
        if len(y_train_city.unique()) > 1:
            # Train the Random Forest Classifier
            rf_classifier = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
            rf_classifier.fit(X_train_scaled, y_train_city)

            # Scale the test data using the same scaler
            X_test_scaled = scaler.transform(X_test)

            # Make predictions for the test set
            predictions = rf_classifier.predict(X_test_scaled)

            # Assign predictions to the correct indices in the original DataFrame
            predicted_growth_values.loc[X_test.index] = predictions  # Correctly assign predictions to test set rows
        else:
            print(f"Skipping city: {city_name} due to insufficient class variation in the target.")
            continue
    else:
        # Skip the group if it has only one class or one sample
        print(f"Skipping city: {city_name} due to insufficient class variation or sample size.")
        continue

# You can now inspect or save the results
opera['predicted_city_growth'] = predicted_growth_values
df_city_growth_sorted = opera[['iso', 'city', 'predicted_city_growth']].sort_values(by='predicted_city_growth', ascending=False)

# Drop duplicates and keep the row with the highest growth for each city
df_opera_city_growth_unique = df_city_growth_sorted.drop_duplicates(subset='city', keep='first')

# Get top 10 cities with highest predicted growth
top_cities_predicted = df_opera_city_growth_unique.head(10)
print(top_cities_predicted)